In [1]:
!pip install tensorflow-datasets

In [2]:
import tensorflow_datasets as tfds
import os
import zipfile
import numpy as np
from PIL import Image


In [4]:
"""
On running this cell , we well get the
  - Episode Metadata
  - Joint Positions
  - Cartesian Positions
  - Left Images
  - Right Images
"""

ds = tfds.load("droid_100", data_dir="gs://gresearch/robotics", split="train")

output_file = "joint_ps.txt"
cartesian_file = "cart_pos.txt"
image_folder = "images_right"
image_left_folder = "images_left"
zip_file_1 = "images_left.zip"
zip_file_2 = "images_right.zip"

"""
I think there are 100 scenes in the RLDS => so any scene_no below that should work => But check that out once pls ..
"""

scene_no = int(input("Enter the scene number to process: "))

os.makedirs(image_folder, exist_ok=True)
os.makedirs(image_left_folder, exist_ok=True)

image_count_1 = 0
image_count_2 = 0

with open(output_file, "w") as joint_file, open(cartesian_file, "w") as cartesian_file:
    for episode_index, episode in enumerate(ds):
        if episode_index == scene_no:
            metadata = episode["episode_metadata"]
            joint_file.write("Episode Metadata:\n")
            joint_file.write(f"{metadata}\n\n")
            print("Episode Metadata:")
            print(metadata)

            joint_file.write("Joint Positions:\n")
            cartesian_file.write("Cartesian Positions:\n")
            for step_index, step in enumerate(episode["steps"]):
                # Joint positions
                joint_position = step["observation"]["joint_position"].numpy()
                joint_file.write(f"{joint_position.tolist()}\n")

                # Cartesian position
                cartesian_position = step["observation"]["cartesian_position"].numpy()
                cartesian_file.write(f"{cartesian_position.tolist()}\n")

                # Left images ..
                image_array_1 = step["observation"]["exterior_image_1_left"].numpy()
                image_1 = Image.fromarray(np.uint8(image_array_1))
                image_path_1 = os.path.join(image_folder, f"image_{step_index:05d}.png")
                image_1.save(image_path_1)
                image_count_1 += 1

                # Right images ..
                image_array_2 = step["observation"]["exterior_image_2_left"].numpy()
                image_2 = Image.fromarray(np.uint8(image_array_2))
                image_path_2 = os.path.join(image_left_folder, f"image_{step_index:05d}.png")
                image_2.save(image_path_2)
                image_count_2 += 1
            break

with zipfile.ZipFile(zip_file_1, "w", zipfile.ZIP_DEFLATED) as zipf_1:
    for root, _, files in os.walk(image_folder):
        for file in files:
            zipf_1.write(os.path.join(root, file),
                         os.path.relpath(os.path.join(root, file), image_folder))

with zipfile.ZipFile(zip_file_2, "w", zipfile.ZIP_DEFLATED) as zipf_2:
    for root, _, files in os.walk(image_left_folder):
        for file in files:
            zipf_2.write(os.path.join(root, file),
                         os.path.relpath(os.path.join(root, file), image_left_folder))

print(f"Episode metadata and joint positions have been saved in '{output_file}'.")
print(f"Cartesian positions have been saved in '{cartesian_file}'.")
print(f"{image_count_1} images from 'exterior_image_1_left' saved in '{zip_file_1}'.")
print(f"{image_count_2} images from 'exterior_image_2_left' saved in '{zip_file_2}'.")


Enter the scene number to process: 4
Episode Metadata:
{'file_path': <tf.Tensor: shape=(), dtype=string, numpy=b'/nfs/kun2/datasets/r2d2/r2d2-data-full/TRI/success/2023-09-21/Thu_Sep_21_17:26:41_2023/trajectory.h5'>, 'recording_folderpath': <tf.Tensor: shape=(), dtype=string, numpy=b'/nfs/kun2/datasets/r2d2/r2d2-data-full/TRI/success/2023-09-21/Thu_Sep_21_17:26:41_2023/recordings/MP4'>}
Episode metadata and joint positions have been saved in 'joint_ps.txt'.
Cartesian positions have been saved in '<_io.TextIOWrapper name='cart_pos.txt' mode='w' encoding='utf-8'>'.
213 images from 'exterior_image_1_left' saved in 'images_left.zip'.
213 images from 'exterior_image_2_left' saved in 'images_right.zip'.


In [13]:
"""
The RLDS data doesn't contain any of the intrinsic info readily , so we need to search for that in the RAW data using the episode metadata path ..

From the above cell output :  numpy=b'/nfs/kun2/datasets/r2d2/r2d2-data-full/TRI/success/2023-09-21/Thu_Sep_21_17:26:41_2023/recordings/MP4'

Now , search for this path in the RAW data ..
"""

!gsutil ls gs://gresearch/robotics/droid_raw/1.0.1/TRI/success/2023-09-21/Thu_Sep_21_17:26:41_2023/recordings/SVO/


gs://gresearch/robotics/droid_raw/1.0.1/TRI/success/2023-09-21/Thu_Sep_21_17:26:41_2023/recordings/SVO/14549178.svo
gs://gresearch/robotics/droid_raw/1.0.1/TRI/success/2023-09-21/Thu_Sep_21_17:26:41_2023/recordings/SVO/20252535.svo
gs://gresearch/robotics/droid_raw/1.0.1/TRI/success/2023-09-21/Thu_Sep_21_17:26:41_2023/recordings/SVO/25947356.svo


In [15]:
"""
TO get the extrinsics , we need to download the trajectory.h5 file and then check the content inside it
"""

!gsutil -m cp -r gs://gresearch/robotics/droid_raw/1.0.1/TRI/success/2023-09-21/Thu_Sep_21_17:26:41_2023/trajectory.h5 .

Copying gs://gresearch/robotics/droid_raw/1.0.1/TRI/success/2023-09-21/Thu_Sep_21_17:26:41_2023/trajectory.h5...
/ [1/1 files][  1.2 MiB/  1.2 MiB] 100% Done                                    
Operation completed over 1 objects/1.2 MiB.                                      


In [14]:
# This .svo will contain the intrinsic info (K matrix which we will subsitute before running the motion code)
!gsutil -m cp -r gs://gresearch/robotics/droid_raw/1.0.1/TRI/success/2023-09-21/Thu_Sep_21_17:26:41_2023/recordings/SVO/20252535.svo .

Copying gs://gresearch/robotics/droid_raw/1.0.1/TRI/success/2023-09-21/Thu_Sep_21_17:26:41_2023/recordings/SVO/20252535.svo...
/ [1/1 files][  9.3 MiB/  9.3 MiB] 100% Done                                    
Operation completed over 1 objects/9.3 MiB.                                      


---
